In [14]:
import math
import numpy as np

from PIL import Image
import piexif

import os
import pandas as pd

from pyproj import Proj

In [28]:
metadatafile = '/Users/epi/GB_selection_log_735768.csv'
dirname = '/Volumes/osx1/over/over2/rawdisplay/imgs/'
imagelist = os.listdir(dirname)

def getImageMetadataFile(metadatafile):
        hbcdtatot = pd.read_csv(metadatafile, index_col=0)
        hbcdtatot.index = pd.to_datetime(hbcdtatot.index)
        hbcdta = hbcdtatot.loc[
            hbcdtatot['Imagename'].isin((i.replace('.jpg', '') for i in imagelist))]
        return hbcdtatot

In [29]:
hbcdtatot = getImageMetadataFile(metadatafile)

In [41]:
hbcdtatot.head()

,x,y,xutm,yutm,Imagename,V_Depth,Water_Depth,Altimeter,Fov,Mm_pix,Salinity,Temp,O2,Cdom,Chlorophyll,Turb
2015-06-19 17:23:33.734,514288.08,4.542407e+06,514242.726667,4.542380e+06,201503.20150619.172333734.187377,10.04,64.77,50.88,NaN,NaN,32.9081,10.75,6.96,56.0,138.0,807.0
2015-06-19 17:23:33.913,514381.66,4.542501e+06,514340.040000,4.542477e+06,201503.20150619.172333913.187378,9.90,64.77,50.88,NaN,NaN,32.9083,10.75,6.96,56.0,138.0,807.0
2015-06-19 17:23:34.070,514392.74,4.542513e+06,514351.870000,4.542489e+06,201503.20150619.172334070.187379,9.90,64.77,50.88,NaN,NaN,32.9083,10.75,6.96,56.0,138.0,807.0
2015-06-19 17:23:34.240,514403.82,4.542524e+06,514363.700000,4.542501e+06,201503.20150619.172334240.187380,9.71,64.77,51.91,NaN,NaN,32.9100,10.75,6.96,48.0,140.0,555.0
2015-06-19 17:23:34.420,514415.09,4.542536e+06,514376.148571,4.542514e+06,201503.20150619.172334420.187381,10.00,64.77,51.91,NaN,NaN,32.9081,10.75,6.96,48.0,140.0,555.0


In [34]:
myProj = Proj("+proj=utm +no_defs +zone=19 +a=6378137 +rf=298.257223563 +towgs84=0.000,0.000,0.000 +to_meter=1")

lon, lat = myProj(hbcdtatot['x'][0], hbcdtatot['y'][0], inverse=True)

In [53]:
def deg2ses(decdeg):
    degrees = int(np.abs(decdeg))
    minutes = int(((np.abs(decdeg)) - degrees)*60)
    seconds = ((np.abs(decdeg) - degrees)*60 - minutes )*60
    return (degrees, minutes, seconds)

def geotag(img, lat, lon, time, date):
    latitude = deg2ses(lat)
    longitude = deg2ses(lon)
    i = Image.open(img)
    GPS = {0: (2, 2, 0, 0), 
           2: ((latitude[0], 1), (latitude[1], 1), (latitude[2], 1000)),
           4: ((longitude[0], 1), (longitude[1], 1), (longitude[2], 1000)),
           5: 1,
           6: (3420199, 1000),
           7: ((time[0], 1), (time[1], 1), (time[2], 1)),
           16: b'M',
           17: (281, 1),
           27: b'ASCII\x00\x00\x00GPS',
           29: date}
    if lat < 0:
        GPS[1]=b'S'
    if lon < 0:
        GPS[3]=b'W'    
    exif_dict = piexif.load(im.info["exif"])
    exif_dict['GPS'] = exif_dict['GPS']
    exif_bytes = piexif.dump(exif_dict)
    i = i.rotate(90, expand=True)
    i.save(img, exif=exif_bytes)


In [54]:
geotag("/Users/epi/Desktop/201503.20150619.181143660.204645 2.jpg", 43.123, -100.23, (10,56,3), '2014:07:11')

In [63]:
exif_dict = piexif.load("/Users/epi/Desktop/201503.20150619.181143660.204645 2.jpg")
if piexif.ImageIFD.MakerNoteSafety in exif_dict["0th"]:
    print("MakerNote is ", exif_dict["0th"][piexif.ImageIFD.MakerNoteSafety])

In [62]:
exif_dict["0th"][piexif.ImageIFD.MakerNoteSafety]='some notes'

In [64]:
exif_dict

{'0th': {256: 3264,
  257: 1836,
  271: b'samsung',
  272: b'SM-T705',
  274: 6,
  282: (72, 1),
  283: (72, 1),
  296: 2,
  305: b'T705DDU1CPI1',
  306: b'2016:12:25 14:33:42',
  531: 1,
  34665: 235,
  34853: 647},
 '1st': {},
 'Exif': {33437: (24, 10),
  34850: 2,
  36864: b'0220',
  36867: b'2016:12:25 14:33:42',
  36868: b'2016:12:25 14:33:42',
  37381: (253, 100),
  37383: 2,
  37385: 0,
  37386: (34, 10),
  37500: b'\x07\x00\x01\x00\x07\x00\x04\x00\x00\x000100\x02\x00\x04\x00\x01\x00\x00\x00\x00 \x01\x00\x0c\x00\x04\x00\x01\x00\x00\x00\x00\x00\x00\x00\x10\x00\x05\x00\x01\x00\x00\x00Z\x00\x00\x00@\x00\x04\x00\x01\x00\x00\x00\x00\x00\x00\x00P\x00\x04\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x03\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  37891: (90, 1),
  40961: 1,
  40962: 3264,
  40963: 1836,
  41986: 0,
  41987: 0,
  41989: 32,
  41990: 3,
  42016: b'E08LSIC04SK'},
 'GPS': {0: (2, 2, 0, 0),
  1: b'N',
  2: ((39, 1), (28, 1), (58748